In [1]:
# general imports
import pathlib
import os
import cobra
from tqdm import tqdm

In [2]:
# ****************************************************************************************************
# Step 5:   Adding gene & reaction annotations back to ftINIT models
# ****************************************************************************************************

# inputs

# path to directory containing output ftINIT models (step 4) in MATLAB format
model_dir = pathlib.Path(os.getcwd()).parent / "models"

# path to Human1 model in SBML format
human1_path = model_dir / "Human-GEM.xml"

In [3]:
# ****************************************************************************************************
# start of the main part of the script

# load Human-GEM annotations
rxn_annotations = {}
gene_annotations = {}
with cobra.io.read_sbml_model(human1_path) as ihuman:
    for rxn in ihuman.reactions:
        rxn_annotations[rxn.id] = ihuman.reactions.get_by_id(rxn.id).annotation
    for gene in ihuman.genes:
        gene_annotations[gene.id] = ihuman.genes.get_by_id(gene.id).annotation

In [4]:
# iterate over files in the models folder and check for files that have not been translated to xml yet
for model_file in model_dir.iterdir():
    if model_file.name.endswith(".mat"):
        xml_name = model_file.name.replace(".mat", ".xml")
        if not (model_dir / xml_name).is_file():
            print(f"Currently re-annotating: {model_file.name}")
            print("*loading model*")
            with cobra.io.load_matlab_model(model_file) as model:
                # iterating over reactions in the model and assigning the respective gene & reaction annotations lost in ftINIT
                print("*updating reaction annotations*")
                for reaction in tqdm(model.reactions):
                    model.reactions.get_by_id(reaction.id).annotation.update(rxn_annotations[reaction.id])
                print("*updating gene annotations*")
                for gene in tqdm(model.genes):
                    model.genes.get_by_id(gene.id).annotation.update(gene_annotations[gene.id])
                print("*saving model*")
                cobra.io.write_sbml_model(model, model_dir / xml_name)
            print(f"Created model: {xml_name}\n")

print(f"All models in the folder {model_dir} are annotated.")

Currently re-annotating: melanocyte_1+0_thrd_model.mat
*loading model*
*updating reaction annotations*


100%|██████████| 8576/8576 [00:00<00:00, 425709.82it/s]


*updating gene annotations*


100%|██████████| 2413/2413 [00:00<00:00, 238992.53it/s]

*saving model*


...

100%|██████████| 8008/8008 [00:00<00:00, 1204647.67it/s]


*updating gene annotations*


100%|██████████| 2413/2413 [00:00<00:00, 343394.14it/s]

*saving model*


Created model: melanocyte_1+1_thrd_model.xml

All models in the folder c:\Users\phili\OneDrive - uni-bielefeld.de\Uni\SoSe24\BA\thesis_scripts\models are annotated.
